In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')

In [100]:

from pyspark import SparkContext
from pyspark.sql import SparkSession
import time
from pyspark.sql.functions import col, count, max

In [3]:

spark = SparkSession \
    .builder \
    .appName("Python Spark Agg Compras") \
    .getOrCreate()

In [4]:

# spark is an existing SparkSession
df = spark.read.option("delimiter", ";").csv("compras.txt", inferSchema = True, header = True)

In [5]:
df.show()

+--------------------+--------+--------------+-----------------+------+----------+
|                Data|    Hora|        Cidade|          Produto| Valor|FormaPagto|
+--------------------+--------+--------------+-----------------+------+----------+
|2015-01-01 00:00:...|09:00:00|     Sao Paulo|Roupas masculinas|214.05|      Amex|
|2015-01-01 00:00:...|09:00:00|Rio de Janeiro| Roupas femininas|153.57|      Visa|
|2015-01-01 00:00:...|09:00:00|      Curitiba|           Musica| 66.08|  Dinheiro|
|2015-01-01 00:00:...|09:00:00|Belo Horizonte|              Pet|493.51|      Visa|
|2015-01-01 00:00:...|09:00:00|       Aracaju|  Roupas infantis|235.63|MasterCard|
|2015-01-01 00:00:...|09:00:00|      Salvador|Roupas masculinas|247.18|MasterCard|
|2015-01-01 00:00:...|09:00:00|      Campinas|          Cameras| 379.6|      Visa|
|2015-01-01 00:00:...|09:00:00|     Sao Paulo|      Eletronicos| 296.8|  Dinheiro|
|2015-01-01 00:00:...|09:00:00|      Londrina|       Brinquedos| 25.38|      Visa|
|201

In [75]:
group = df.groupBy(['Cidade', 'Produto'])

In [143]:
count_prod = group.count()

+--------------+-----------------+-----+
|        Cidade|          Produto|count|
+--------------+-----------------+-----+
|Belo Horizonte|  Roupas infantis| 1072|
|         Serra|      Video Games|  540|
|      Brasilia|      Eletronicos|  544|
|       Mossoro|   Saude e beleza|  545|
|        Sumare|     Computadores|  561|
|   Sete Lagoas|          Cameras|  543|
|      Sorocaba|      Eletronicos|  532|
|      Londrina|Roupas masculinas| 2107|
|      Teresina|  Roupas infantis| 1089|
|     Fortaleza|  Roupas infantis| 1097|
| Foz do Iguacu|           Livros|  527|
|      Sao Jose|  Roupas infantis| 1123|
| Nova Friburgo|Roupas masculinas|  552|
|       Chapeco|       Brinquedos|  548|
|Belo Horizonte|       Brinquedos|  562|
|     Rio Claro|   Saude e beleza|  526|
|      Ourinhos|             DVDs|  529|
|     Sao Paulo|              CDs| 3358|
|       Niteroi|  Roupas infantis| 1053|
|Belo Horizonte|       Artesanato|  559|
+--------------+-----------------+-----+
only showing top

In [138]:
max_prod = count_prod.groupBy('Cidade').max('count').select(col('Cidade'), col("max(count)").alias("count"))

In [136]:
max_prod.show()

+-------------------+----+
|             Cidade| max|
+-------------------+----+
|       Juiz de Fora|1089|
|          Americana|1072|
|          Fortaleza|1097|
|             Santos|2089|
|            Uberaba|1087|
|           Dourados|1096|
|           Curitiba|1026|
|       Campo Grande|1055|
|        Barra Mansa|1105|
|            Maringa|2177|
|Presidente Prudente|1062|
|             Palmas|1128|
|           Sao Jose|1123|
|           Criciuma|1057|
|             Sobral|1101|
|              Cotia|1079|
|             Cuiaba|1083|
|          Guarulhos|2128|
|        Sete Lagoas|1034|
|     Angra dos Reis|1108|
+-------------------+----+
only showing top 20 rows



In [147]:
df2 = count_prod.join(max_prod, ['Cidade', 'count'])

In [148]:
oder = df2.orderBy('Cidade')

In [149]:
oder.show()

+--------------+-----+---------------+
|        Cidade|count|        Produto|
+--------------+-----+---------------+
|      Alvorada| 1089|Roupas infantis|
|     Americana| 1072|Roupas infantis|
|Angra dos Reis| 1108|Roupas infantis|
|       Aracaju| 1096|Roupas infantis|
|     Aracatuba| 1052|Roupas infantis|
|   Barra Mansa| 1105|Roupas infantis|
|       Barueri| 1084|Roupas infantis|
|         Bauru| 1093|Roupas infantis|
|         Belem| 1075|Roupas infantis|
|Belo Horizonte| 1072|Roupas infantis|
|     Boa Vista| 1087|Roupas infantis|
|      Brasilia| 1078|Roupas infantis|
|     Cabo Frio| 2125|Roupas infantis|
|      Campinas| 2229|Roupas infantis|
|  Campo Grande| 1055|Roupas infantis|
|      Cascavel| 1070|Roupas infantis|
|        Caxias| 1079|Roupas infantis|
|       Chapeco| 1120|Roupas infantis|
|       Colombo| 3252|Roupas infantis|
|         Cotia| 1079|Roupas infantis|
+--------------+-----+---------------+
only showing top 20 rows

